In [14]:
!pip3 install ddgs

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=35790) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


  Using cached ddgs-9.6.0-py3-none-any.whl.metadata (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.7/815.7 kB 1.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python3.12 -m pip install --upgrade pip


In [24]:
# moderator_app.py
from __future__ import annotations
from dataclasses import dataclass, field
from typing import List, Optional, Dict, Literal

from dotenv import load_dotenv
load_dotenv()  # loads OPENAI_API_KEY from .env

from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from ddgs import DDGS

# -----------------------------
# 1) STATE (shared memory)
# -----------------------------
@dataclass
class DebateCitation:
    title: str
    url: str
    snippet: Optional[str] = None

@dataclass
class DebateMessage:
    content: str
    key_points: List[str] =  field(default_factory=list)
    citations: List[DebateCitation] = field(default_factory=list)

# @dataclass
# class JudgeScores:
#     clarity_structure: int = Field(ge=1, le=5)
#     evidence_examples: int = Field(ge=1, le=5)
#     responsiveness: int = Field(ge=1, le=5)
#     persuasiveness_creativity: int = Field(ge=1, le=5)

@dataclass
class JudgeMessage:
    decision: Literal["pro", "con", "tie"]
    pro_score: Dict[str, int]
    con_score: Dict[str, int]
    score_justification: str
    rationale: str
    pro_feedback: str
    con_feedback: str
    key_points: List[str] = field(default_factory=list)

@dataclass
class DebateHistory:
    role: Literal["pro", "con", "judge"]
    phase: Literal["opening", "rebuttal", "closing", "decision"]
    output: DebateMessage

@dataclass
class DebateState:
    topic: str
    status: str = "init"                       # "init" | "rejected" | "ready"
    safety_flags: List[Dict] = field(default_factory=list)  # audit trail
    message: Optional[str] = None              # explanation / next-step text
    history: List[DebateHistory] = field(default_factory=list)  # past messages
    use_search: bool = True


# -----------------------------
# 2) SCHEMA for structured output
# -----------------------------
Action = Literal["allow", "rephrase", "reject"]

class ModerationResult(BaseModel):
    action: Action
    categories: List[str] = Field(default_factory=list)     # e.g., ["illegal_activity", "hate"]
    explanation: str                                        # short human-safe rationale
    safe_topic: Optional[str] = None                        # required if action == "rephrase"
    notes: List[str] = Field(default_factory=list)          # optional extra guidance

# Use a Pydantic model for structured LLM output, then map to dataclass
class DebateCitationModel(BaseModel):
    title: str
    url: str
    snippet: Optional[str] = None

class DebateMessageModel(BaseModel):
    content: str = Field(..., max_length=2000)  # Limit content length
    key_points: List[str] = Field(default_factory=list, max_length=3)  # Limit key points
    citations: List[DebateCitationModel] = Field(default_factory=list, max_length=2)  # Limit citations

class JudgeScoresModel(BaseModel):
    clarity_structure: int = Field(..., ge=1, le=5)
    evidence_examples: int = Field(..., ge=1, le=5)
    responsiveness: int = Field(..., ge=1, le=5)
    persuasiveness_creativity: int = Field(..., ge=1, le=5)

class JudgeMessageModel(BaseModel):
    decision: Literal["pro", "con", "tie"]
    pro_score: JudgeScoresModel
    con_score: JudgeScoresModel
    score_justification: str = Field(
        ..., 
        description="Explain the key differences in scores between Pro and Con",
        max_length=1000
    )
    rationale: str = Field(..., max_length=1000)
    pro_feedback: str = Field(..., max_length=500)
    con_feedback: str = Field(..., max_length=500)
    key_points: List[str] = Field(default_factory=list, max_length=3)


# -----------------------------
# 3) LLM setup with token limits
# -----------------------------
llm = ChatOpenAI(
    model="gpt-4o-mini", 
    temperature=0,
    max_tokens=2000,  # Set a reasonable max_tokens limit
    request_timeout=30  # seconds
)

# -----------------------------
# 4) NODES
# -----------------------------
def moderator_node(state: DebateState) -> DebateState:
    """Classify a topic as allow / rephrase / reject; optionally rephrase it."""
    # 4a) quick keyword short-circuit (fast, local)
    quick_block = [
        "build a bomb", "make a bomb", "child sexual", "sexual content with minors",
        "self-harm instructions", "how to hack", "credit card skimming",
        "assassinate", "terrorist", "make a gun", "kill"
    ]
    topic_l = (state.topic or "").strip().lower()
    if any(k in topic_l for k in quick_block):
        res = ModerationResult(
            action="reject",
            categories=["illegal_or_extreme_harm"],
            explanation="Disallowed: violent/illegal instruction content.",
            safe_topic=None,
            notes=["Reframe as an ethics/policy discussion rather than instructions."]
        )
        state.safety_flags.append(res.model_dump())
        return state

    # 4b) LLM moderation + academic reframe
    system = (
        "You are a safety moderator for a debate platform. "
        "Decide one of: allow | rephrase | reject. "
        "Disallow topics with step-by-step illegal harm, sexual content with minors, "
        "self-harm instructions, targeted hate/violence, invasive privacy breaches, malware creation, etc. "
        "Also don't allow topics that have any relation to religion or politics, the topics should all be "
        "leaning more towards being lighthearted and less controversial. "
        "If rephrase: produce a neutral, academic framing (policy/ethics/history) that preserves learning value. "
        "Also ensure that the rephrase is detailed and provides two clear sides to argue about. "
        "Keep your response concise. Return ONLY structured object matching the provided schema."
    )
    user = f'Topic: "{state.topic}"'

    mod_llm = llm.with_structured_output(schema=ModerationResult)
    result: ModerationResult = mod_llm.invoke(f"{system}\n\n{user}")

    # audit trail
    state.safety_flags.append(result.model_dump())

    # apply rephrase if given
    if result.action == "rephrase" and result.safe_topic:
        state.topic = result.safe_topic

    return state

def moderator_router(state: DebateState) -> str:
    """Decide which node to run after moderator."""
    last = state.safety_flags[-1] if state.safety_flags else {}
    action = last.get("action")
    if action == "reject":
        return "reject_exit"
    return "proceed"  # allow OR rephrase both continue

def reject_exit(state: DebateState) -> DebateState:
    state.status = "rejected"
    last = state.safety_flags[-1] if state.safety_flags else {}
    state.message = f"❌ Rejected. Reason: {last.get('explanation','')}"
    return state

def proceed_node(state: DebateState) -> DebateState:
    state.status = "ready"
    last = state.safety_flags[-1] if state.safety_flags else {}
    if last.get("action") == "rephrase":
        state.message = f"✅ Rephrased to safe topic: {state.topic}. Ready to start the debate flow."
    else:
        state.message = f"✅ Allowed topic: {state.topic}. Ready to start the debate flow."
    return state

# ---------- Web search function (DuckDuckGo) ----------
# --- DuckDuckGo search helper ---
def ddg_search(query: str, *, max_results: int = 4, region: str = "uk-en") -> List[DebateCitation]:
    """
    Runs a DuckDuckGo text search and returns DebateCitation objects.
    Safesearch is 'moderate' to avoid unsafe content in debate references.
    """
    citations: List[DebateCitation] = []
    try:
        with DDGS(timeout=10) as ddgs:
            for r in ddgs.text(
                query,
                region=region,
                safesearch="moderate",
                max_results=max_results,
            ):
                title = r.get("title") or r.get("source") or "Untitled"
                url = r.get("href") or r.get("url")
                snippet = r.get("body") or r.get("snippet") or ""
                if url:
                    citations.append(DebateCitation(title=title, url=url, snippet=snippet))
    except Exception as e:
        # Keep the debate running even if search fails
        print(f"[warn] ddg_search failed: {e}")
    return citations



# ---------- PRO / CON NODES ----------
def pro_node(state: DebateState) -> DebateState:
    system = (
        "You are an expert debater arguing IN FAVOR of the position. "
        "Your goal is to REBUT the opposing argument while advancing NEW points. "
        "\n\nRules:"
        "\n1. DIRECTLY address and refute at least one of Con's specific claims"
        "\n2. Introduce at least ONE new supporting argument not yet discussed"
        "\n3. Use different examples than before"
        "\n4. Keep response to 150-200 words"
        "\n5. DO NOT make up citations or fake sources"
        "\n6. DO NOT just repeat your earlier points"
        "\n7. Show why Con's logic is flawed or incomplete"
    )
    mod_llm = llm.with_structured_output(schema=DebateMessageModel)

    # --- NEW: gather web evidence (short, credible-leaning) ---
    citations: List[DebateCitation] = []
    if getattr(state, "use_search", False):
        # Nudge towards credible sources; tweak as you like
        q = f'{state.topic} (statistics OR study OR report) site:.gov OR site:.edu OR site:.org'
        citations = ddg_search(q, max_results=3)

    if not state.history:
        prompt = (
            f"Topic: {state.topic}\n\n"
            f"Provide a concise opening argument supporting this position.\n"
        )
    else:
        prior_text = state.history[-1].output.content
        prompt = (
            f"Topic: {state.topic}\n\n"
            f"Previous opposing argument: {prior_text}\n\n"
            f"Provide a concise rebuttal supporting your position."
        )
        # Optional: discourage repetition
        if len(state.history) >= 2:
            prev_kps = state.history[-2].output.key_points
            if prev_kps:
                prompt += f"\nDo not repeat your earlier key points: {', '.join(prev_kps)}"

    # Add brief evidence snippets (helps model cite correctly without huge context)
    if citations:
        snippet_block = "\n".join(
            f"- {c.title}: {c.snippet[:220]} ({c.url})" for c in citations
        )
        prompt += f"\n\nUse these sources if relevant:\n{snippet_block}"

    result: DebateMessageModel = mod_llm.invoke(f"{system}\n\n{prompt}")

    # If model didn't emit citations, backfill from DDG; otherwise keep model’s
    out_citations = (
        [DebateCitation(**c.model_dump()) for c in result.citations]
        if result.citations else citations[:2]  # keep it short
    )

    msg = DebateMessage(
        content=result.content,
        key_points=result.key_points,
        citations=out_citations,
    )
    phase = "opening" if not state.history else "rebuttal"
    state.history.append(DebateHistory(role="pro", phase=phase, output=msg))
    return state


def con_node(state: DebateState) -> DebateState:
    system = (
        "You are an expert debater arguing AGAINST the position. "
        "Your goal is to REBUT the opposing argument while advancing NEW counterpoints. "
        "\n\nRules:"
        "\n1. DIRECTLY address and refute at least one of Pro's specific claims"
        "\n2. Introduce at least ONE new opposing argument not yet discussed"
        "\n3. Use different examples than before"
        "\n4. Keep response to 150-200 words"
        "\n5. DO NOT make up citations or fake sources"
        "\n6. DO NOT just repeat your earlier points"
        "\n7. Show why Pro's reasoning is flawed, incomplete, or overlooks important considerations"
    )
    mod_llm = llm.with_structured_output(schema=DebateMessageModel)

    citations: List[DebateCitation] = []
    if getattr(state, "use_search", False):
        q = f'{state.topic} criticisms OR limitations OR risks site:.gov OR site:.edu OR site:.org'
        citations = ddg_search(q, max_results=3)

    prior_text = state.history[-1].output.content if state.history else ""
    if state.history and state.history[-1].phase == "opening":
        prompt = (
            f"Topic: {state.topic}\n\n"
            f"Previous Pro argument: {prior_text}\n\n"
            f"Provide a concise rebuttal opposing this position."
        )
        phase = "rebuttal"
    else:
        prompt = (
            f"Topic: {state.topic}\n\n"
            f"Previous argument: {prior_text}\n\n"
            f"Provide a concise closing argument opposing this position."
        )
        if len(state.history) >= 2:
            prev_kps = state.history[-2].output.key_points
            if prev_kps:
                prompt += f"\nDo not repeat your earlier key points: {', '.join(prev_kps)}"
        phase = "closing"

    if citations:
        snippet_block = "\n".join(
            f"- {c.title}: {c.snippet[:220]} ({c.url})" for c in citations
        )
        prompt += f"\n\nUse these sources if relevant:\n{snippet_block}"

    result: DebateMessageModel = mod_llm.invoke(f"{system}\n\n{prompt}")

    out_citations = (
        [DebateCitation(**c.model_dump()) for c in result.citations]
        if result.citations else citations[:2]
    )

    msg = DebateMessage(
        content=result.content,
        key_points=result.key_points,
        citations=out_citations,
    )
    state.history.append(DebateHistory(role="con", phase=phase, output=msg))
    return state


def judge_node(state: DebateState) -> DebateState:
    """Generate judge evaluation with strict length limits."""
    system = (
    "You are a CRITICAL Judge in a debate competition. Your role is to identify clear differences "
    "between the debaters and declare a winner. Ties should be RARE (only when truly deadlocked).\n\n"
    "Evaluate both sides on these 4 criteria (1-5 scale):\n\n"
    "1. **Clarity & Structure** (1=confusing, 3=adequate, 5=crystal clear)\n"
    "2. **Use of Evidence** (1=none/weak, 3=some examples, 5=compelling evidence)\n"
    "3. **Responsiveness** (1=ignores opponent, 3=partial engagement, 5=direct rebuttals)\n"
    "4. **Persuasiveness** (1=unconvincing, 3=mixed, 5=highly persuasive)\n\n"
    "CRITICAL INSTRUCTIONS:\n"
    "- Use the FULL range of scores (1-5). Don't cluster everything around 3-4.\n"
    "- Identify specific strengths and weaknesses that differentiate the debaters.\n"
    "- Pick a winner unless the debate is genuinely deadlocked.\n"
    "- A 'tie' should only occur if total scores are within 1-2 points AND no clear victor emerges.\n"
    "- Be decisive but fair. Look for even small advantages.\n"
    )
    
    # Summarize debate history concisely to avoid token overflow
    debate_summary = []
    for h in state.history:
        debate_summary.append(f"{h.role.upper()} ({h.phase}): {h.output.content}")
    
    # Add temperature to make the model less deterministic
    mod_llm = llm.with_structured_output(schema=JudgeMessageModel, temperature=0.7)

    # Or add to your prompt:
    prompt = (
        f"Topic: {state.topic}\n\n"
        f"Debate Summary:\n" + "\n\n".join(debate_summary) + "\n\n"
        f"IMPORTANT: Carefully evaluate differences between Pro and Con. "
        f"Declare a winner unless they are truly equal. Provide your judgment."
    )
    
    result: JudgeMessageModel = mod_llm.invoke(f"{system}\n\n{prompt}")
    
    msg = JudgeMessage(
        decision=result.decision,
        pro_score=result.pro_score.model_dump(),
        con_score=result.con_score.model_dump(),
        score_justification=result.score_justification,
        rationale=result.rationale,
        pro_feedback=result.pro_feedback,
        con_feedback=result.con_feedback,
        key_points=result.key_points
    )
    state.history.append(DebateHistory(role="judge", phase="decision", output=msg))
    return state

def print_scores_with_bars(pro_score: Dict[str, int], con_score: Dict[str, int]):
    print("\n📊 DEBATE SCORES")
    print("="*70)
    
    criteria = {
        'clarity_structure': 'Clarity & Structure',
        'evidence_examples': 'Evidence & Examples',
        'responsiveness': 'Responsiveness',
        'persuasiveness_creativity': 'Persuasiveness'
    }
    
    for key, name in criteria.items():
        pro = pro_score.get(key, 0)
        con = con_score.get(key, 0)
        
        pro_bar = "█" * pro + "░" * (5 - pro)
        con_bar = "█" * con + "░" * (5 - con)
        
        print(f"\n{name}")
        print(f"  Pro: {pro_bar} {pro}/5")
        print(f"  Con: {con_bar} {con}/5")
    
    print("\n" + "="*70)
    print(f"TOTAL: Pro {sum(pro_score.values())}/20  |  Con {sum(con_score.values())}/20")
    print("="*70 + "\n")


# ---------- FLOW ROUTER ----------
def debate_flow_router(state: DebateState) -> str:
    """
    Turn order:
      - (no history) -> pro (opening)
      - after pro opening -> con (rebuttal)
      - after con rebuttal -> pro (rebuttal)
      - after pro rebuttal -> con (closing)
      - after con closing -> judge
      - after judge -> END
    """
    if not state.history:
        return "pro"

    last = state.history[-1]
    if last.role == "pro" and last.phase in ("opening", "rebuttal"):
        return "con"

    if last.role == "con":
        if last.phase == "rebuttal":
            return "pro"
        if last.phase == "closing":
            return "judge"
    
    if last.role == "judge":
        return "__end__"

    # Fallback
    return "__end__"


# -----------------------------
# 5) GRAPH WIRING
# -----------------------------
graph = StateGraph(DebateState)

# Existing nodes
graph.add_node("moderator", moderator_node)
graph.add_node("reject_exit", reject_exit)
graph.add_node("proceed", proceed_node)

# New debate nodes
graph.add_node("pro", pro_node)
graph.add_node("con", con_node)
graph.add_node("judge", judge_node)

graph.set_entry_point("moderator")

# Moderator decides whether to proceed or reject
graph.add_conditional_edges(
    "moderator",
    moderator_router,
    {"reject_exit": "reject_exit", "proceed": "proceed"}
)

graph.add_edge("reject_exit", END)

# Enter debate flow after proceed
graph.add_conditional_edges(
    "proceed",
    debate_flow_router,
    {"pro": "pro", "con": "con", "__end__": END}
)

# Keep routing between pro/con until closing -> END
graph.add_conditional_edges(
    "pro",
    debate_flow_router,
    {"pro": "pro", "con": "con", "__end__": END}
)
graph.add_conditional_edges(
    "con",
    debate_flow_router,
    {"pro": "pro", "con": "con", "judge": "judge", "__end__": END}
)
graph.add_conditional_edges(
    "judge",
    debate_flow_router,
    {"__end__": END}
)

app = graph.compile()

# -----------------------------
# 6) CLI entrypoint
# -----------------------------
if __name__ == "__main__":
    try:
        topic = input("Enter a debate topic: ")
    except KeyboardInterrupt:
        raise SystemExit

    initial = DebateState(topic=topic)

    print("\n--- Live Debate ---")
    final_state = None

    for event in app.stream(initial):  
        for node_name, payload in event.items():
            # In default mode, payload is a dict with metadata (value, logs, etc.)
            state = payload.get("value", payload)

            if node_name == "moderator":
                flags = state.get("safety_flags", [])
                last = flags[-1] if flags else {}
                action = last.get("action")
                print("\n[MODERATOR]")
                print("Topic:", state.get("topic"))
                if action == "reject":
                    print("Decision: REJECT")
                    print("Reason:", last.get("explanation", ""))
                elif action == "rephrase":
                    print("Decision: REPHRASE")
                    print("Safe Topic:", state.get("topic"))
                    print("Why:", last.get("explanation", ""))
                else:
                    print("Decision: ALLOW")

            elif node_name in ("pro", "con"):
                hlist = state.get("history", [])
                if not hlist:
                    continue
                h = hlist[-1]  # DebateHistory dataclass
                role = h.role.upper()
                phase = h.phase.capitalize()
                out = h.output  # DebateMessage dataclass
                print(f"\n[{role} - {phase}]")
                print(out.content)
                if out.key_points:
                    print("Key Points:", ", ".join(out.key_points))
                if out.citations:
                    print("Citations:")
                    for c in out.citations:
                        snip = f" - {c.snippet}" if c.snippet else ""
                        print(f" • {c.title} ({c.url}){snip}")

            elif node_name == "judge":
                hlist = state.get("history", [])
                if not hlist:
                    continue
                h = hlist[-1]
                out = h.output  # JudgeMessage dataclass
                print("\n[JUDGE DECISION]")
                print("Decision:", getattr(out, "decision", "").upper())
                print_scores_with_bars(out.pro_score, out.con_score)
                print("Score Justification:", getattr(out, "score_justification", ""))
                print("Rationale:", getattr(out, "rationale", ""))
                pf = getattr(out, "pro_feedback", "")
                cf = getattr(out, "con_feedback", "")
                if pf:
                    print("Pro Feedback:", pf)
                if cf:
                    print("Con Feedback:", cf)
                kps = getattr(out, "key_points", []) or []
                if kps:
                    print("Key Points:", ", ".join(kps))

            final_state = state  # keep latest

    if final_state:
        print("\n--- Debate Complete ---")
        print("Final Topic:", final_state.get("topic"))


--- Live Debate ---

[MODERATOR]
Topic: data privacy
Decision: ALLOW

[PRO - Opening]
While Con argues that data privacy regulations stifle innovation, this perspective overlooks the fact that robust data privacy can actually foster trust and encourage user engagement. For instance, companies like Apple have thrived by prioritizing user privacy, demonstrating that consumers are willing to pay a premium for products that protect their data. This contradicts Con's claim that privacy measures hinder business growth.

Moreover, a new supporting argument is the increasing prevalence of data breaches, which not only compromise individual privacy but also lead to significant financial losses for businesses. According to a report by IBM, the average cost of a data breach in 2023 was $4.45 million. This highlights that without stringent data privacy measures, companies face greater risks that can ultimately stifle innovation and growth.

In conclusion, prioritizing data privacy is not just a r